In [3]:
using Pkg
Pkg.activate("..")

using MolecularGraph, MolecularGraph.Graphs

  Activating project at `~/.julia/dev/MOGMOG.jl`


In [14]:
# Tryptophan
supertypes(typeof(MolecularGraph.smilestomol("CC(C)(C)C")))

(SMILESMolGraph, SimpleMolGraph{Int64, SMILESAtom, SMILESBond}, AbstractMolGraph{Int64}, Graphs.AbstractGraph{Int64}, Any)

In [5]:
function dfs_with_backtrack(g::MolGraph, start::Int) # dfs, depth first search - trädsöknings metod 
    n = nv(g) # antalet atomer i grafen sparas i n 
    visited = falses(n) # en vektor visisted som berättar vilka som är besökta och inte. alla får false till en början 
    parent  = fill(0, n)  # en vektor av längd n och alla värden är 0 till att börja med. sparar alltså varje atoms förälder
    
    function _dfs(u) # en inre funktion som djupsöker. Tar in u som argument 
        visited[u] = true # markera u som besökt 
        for v in neighbors(g, u) # kolla efter u grannar 
            if !visited[v] # om grannen ej är besökt 
                parent[v] = u # då har vi nått v genom u, dvs u är föräldern till v 
                _dfs(v) # gör samma för v och alla grannar i grenen 
                # when we return here, we've finished exploring the entire v‐branch
                println("Finished branch at $v → jumping back to $u")
            end
        end
    end
    
    parent[start] = start   # root’s “parent” is itself
    _dfs(start) # börja med den första 
    return parent # returnera föräldar listan 
end

# Example usage:
mg = MolecularGraph.smilestomol("CC(C)(CC)C") # Skapar en MolGraph från SMILES-strängen "CC(C)(C)C".
parents = dfs_with_backtrack(mg, 1) # funktionen returnerar en lista och har tagit in molekylen i mg och startindex 1. 


Finished branch at 3 → jumping back to 2
Finished branch at 5 → jumping back to 4
Finished branch at 4 → jumping back to 2
Finished branch at 6 → jumping back to 2
Finished branch at 2 → jumping back to 1


6-element Vector{Int64}:
 1
 1
 2
 2
 4
 2

In [ ]:
# takes idealized smiles with only capital letters and parentheses
# atom types dont technically matter.
# we assume each atom type is just one character
 # function get_bond_pairs(clean_smiles::String,positions, cutoff=1.2)
    # bond_pairs = Tuple{Int,Int}[]
    # for i in 1:length(clean_smiles)-1:
       #  for j in 1+1:length(clean_smiles):
         #    distance = norm(positions[i] - positions[j])
          #   if distance ≤ cutoff
          #   push!(bond_pairs, (i, j))

  #   end
# end

In [1]:


using Test

In [2]:
# unwanted_characters = collect("0123456789.=#\$:/\\")
unwanted_characters = collect("0123456789.=#\$:/\\")

smiles = "C3C1(C2.:#\$)(C)1C"
clean_smiles = uppercase(replace(smiles, [s => "" for s in unwanted_characters]...))


"CC(C)(C)C"

In [20]:
# försök att få ringar att fungera

function get_bond_pairs(clean_smiles::String)
    bond_pairs   = Tuple{Int,Int}[]   # resultatlista
    stack        = Int[]              # för att spara var grenar startar
    ring_indices = Dict{Char,Int}()   # för att hantera ring-siffror
    last_atom    = 0                  # index på senaste atom
    atom_idx     = 0                  # atom-räknare

    for c in collect(clean_smiles)
        if isuppercase(c)
            atom_idx += 1
            if last_atom != 0
                push!(bond_pairs, (last_atom, atom_idx))
            end
            last_atom = atom_idx

        elseif c == '('
            push!(stack, last_atom)

        elseif c == ')'
            last_atom = pop!(stack)

        elseif isdigit(c)
            # första gång vi ser siffran → spara “ring-start”
            if !haskey(ring_indices, c)
                ring_indices[c] = last_atom
            else
                # andra gången → skapa ringbindningen
                push!(bond_pairs, (last_atom, ring_indices[c]))
                delete!(ring_indices, c)
            end
        end
    end

    return bond_pairs
end


get_bond_pairs (generic function with 1 method)

In [3]:
# första variant 
# Vi utgår från att `clean_smiles` innehåller t.ex. "CC(C)(CC)C"
 function get_bond_pairs(clean_smiles::String)
    bond_pairs = Tuple{Int,Int}[]   # resultatlista
    stack      = Int[]              # för att spara uppgångspunkter vid '('
    last_atom  = 0                  # index på senaste atom i “huvudkedjan”
    atom_idx   = 0                  # räknare för atomer

    for c in collect(clean_smiles)
        if isuppercase(c)           # en atom (antar enkelteckens-atomer)
            atom_idx += 1
            if last_atom != 0
                push!(bond_pairs, (last_atom, atom_idx))
            end
            last_atom = atom_idx
        elseif c == '('
            # sparar var vi “grenade” från
            push!(stack, last_atom)
        elseif c == ')'
            # när grenen är slut, gå tillbaka till det sparade indexet
            last_atom = pop!(stack)
        else
            # (om du hade andra tecken, ignoreras de här)
        end
    end

    return bond_pairs
end


get_bond_pairs (generic function with 1 method)

In [5]:
@test get_bond_pairs("CC(C)(C)C")             == [(1,2),(2,3),(2,4),(2,5)]
@test get_bond_pairs("CC(C)(C)CC")           == [(1,2),(2,3),(2,4),(2,5),(5,6)]
@test get_bond_pairs("CC(C)(CC(C)C)C")       == [(1,2),(2,3),(2,4),(4,5),(5,6),(5,7),(2,8)]

# test passed 

Test Passed

In [9]:
using Test 

In [10]:
unwanted_characters = collect("0123456789.=#\$:/\\")

smiles = "C3C1(C2.:#\$)(C)1C"
clean_smiles = uppercase(replace(smiles, [s => "" for s in unwanted_characters]...))

"CC(C)(C)C"

In [21]:
# CLIMBS FUNCTION ####
# If the atom is by itself and has no () = 0 
# If there is one atom inside the ()= 1
# If there are more than one inside the (), then the last atom will recieve a climb of the amount of atoms in that chain but the other ones will have zero (except if there are more sidechains in that sidechain).
# The last atom in the molecule always get a climb 0 

function compute_climbs(smiles::String)
    # --- Steg 1: Tilldela varje atom ett grupp-ID och samla alla atom-index per grupp ---
    stack = Int[]                           # håller öppna grupp-ID:n
    atom_groups = Int[]                     # grupp-ID per atom (0 = ingen)
    group_members = Dict{Int,Vector{Int}}() # atom-index per grupp
    next_group_id = 1

    for c in smiles
        if c == '('
            # starta ny grupp
            group_members[next_group_id] = Int[]
            push!(stack, next_group_id)
            next_group_id += 1

        elseif c == ')'
            # stäng senaste öppna grupp
            pop!(stack)

        elseif isletter(c)
            # bokstav = atom
            gid = isempty(stack) ? 0 : stack[end]
            push!(atom_groups, gid)
            if gid != 0
                push!(group_members[gid], length(atom_groups))
            end
        end
    end

    # --- Steg 2: Sätt climb endast på sista atom i varje grupp ---
    n = length(atom_groups)
    climbs = zeros(Int, n)
    for (gid, members) in group_members
        S = length(members)
        last_idx = members[end]
        climbs[last_idx] = S
    end

    return climbs
end


compute_climbs (generic function with 1 method)

In [25]:
@test compute_climbs("CC(C)(CC)C")             == [0,0,1,0,2,0]
@test compute_climbs("CC(C)(C)C")             == [0,0,1,1,0]
@test compute_climbs("CC(C)(CC(C)C)C")       == [0,0,1,0,0,1,3,0]
@test compute_climbs("CC(C(C)C)(CC(C)CC)C(C)C")   == [0, 0, 0, 1, 2, 0, 0, 1, 0, 4, 0, 1, 0]

# alla passed 

Test Passed

Error During Test at /Users/alicestenbeck/.julia/dev/MOGMOG.jl/Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:3
  Test threw exception
  Expression: get_bond_pairs("CC(C)(C)C") == [(1, 2), (2, 3), (2, 4), (2, 5)]
  UndefVarError: `get_bond_pairs` not defined in `Main`
  Suggestion: check for spelling errors or missing imports.
  Stacktrace:
   [1] macro expansion
     @ ~/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:676 [inlined]
   [2] top-level scope
     @ ~/.julia/dev/MOGMOG.jl/Notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:3


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [ ]:
# Ändringar och tillägg:
# - Infört två nya tokens ANCHOR_PUSH och ANCHOR_POP för att markera ankar-punkter vid DFS
# - Lagt till anchor_stack för att spara koordinater vid push och pop
# - Byggt atom_seq, coord_seq, atom_mask, coord_mask med ankar-tokens och dummy-koordinater (dvs utfyllnad för där push/pop är)
# - Uppdaterad loss-funktion som maskerar bort ankar-tokens i både atom- och koordinat-förlust
# - Behållt ursprunglig MOG-logpdf-stub och Flux-baserad atomtyp-förlust

using Graphs, MoleculeGraph, Flux, Distributions

# Definiera tokens för ankar-push och ankar-pop (lägg in i er atom_dict)
const ANCHOR_PUSH = 1001    # Token-ID för push
const ANCHOR_POP  = 1002    # Token-ID för pop

# DFS-traversering med explicit push/pop
function dfs_with_anchors(g::MolGraph, coords::Vector{SVector{3,Float32}}, start::Int)
    n = nv(g)                                   # antalet atomer i grafen sparas i n 
    visited      = falses(n)                    # en vektor visisted som berättar vilka som är besökta och inte. alla får false till en början 
    atom_seq     = Int[]                        # Sekvens av atom-ID:n + ankar-tokens. Ska ej ta in dessa saker som argument eftersom vi gör en ny av allt 
    coord_seq    = Vector{SVector{3,Float32}}() # Koordinatsekvens, inkluderar dummy för anchors
    atom_mask    = Bool[]                       # True för riktiga atomer, false för anchors
    coord_mask   = Bool[]                       # True för giltiga displacements
    anchor_stack = SVector{3,Float32}[]         # Sparar koordinater vid push

    function _traverse(u) # Början på den rekursiva hjälpfunktionen _traverse, som tar den aktuella noden/atomen u som argument.
        visited[u] = true # Markera nod u som besökt i vektorn visited, så vi inte återvänder hit igen.
        # Lägg till aktuell atom
        push!(atom_seq, u) 
        push!(coord_seq, coords[u])
        push!(atom_mask, true)
        push!(coord_mask, true)

        # Hitta oexplorerade grannar
        nbrs = [v for v in neighbors(g, u) if !visited[v]] # inte besökta grannar sparas i en lista 
        for v in nbrs # besök de ej besökta 
            if length(nbrs) >= 2 # Gör push om flera grenar. Om det skulle varit 1 finns det bara en väg framåt och är ej en förgrening
                push!(anchor_stack, coords[u]) # Spara aktuell position på stacken
                
                # Lägg till push-token med dummy-/ankar-koordinat för alla dessa eftersom detta sker om det är en förgrening och då ska vi ha push 
                push!(atom_seq, ANCHOR_PUSH)
                push!(coord_seq, coords[u])       # alternativt zeros(SVector{3,Float32})
                push!(atom_mask, false)           # exkludera i atomförlust
                push!(coord_mask, false)          # exkludera i koordinatförlust
            end

            _traverse(v) # Rekursivt gå in i grenen

            # Efter att grenen är utforskad: pop
            if length(nbrs) >= 2
                anchor_pos = pop!(anchor_stack) # Hämta sparad position

                # Lägg till pop-token med återställd ankar-koordinat
                push!(atom_seq, ANCHOR_POP)
                push!(coord_seq, anchor_pos)
                push!(atom_mask, false)
                push!(coord_mask, false)
            end
        end
    end

    _traverse(start) 
    return atom_seq, coord_seq, atom_mask, coord_mask
end

# Uppdaterad loss-funktion som ignorerar ankar-tokens
function loss(model, atom_seq, coord_seq, atom_mask, coord_mask)
    # Bygg indata för modellen
    atom_ids = Flux.onehotbatch(atom_seq, 1:model.vocab_size)  # gör en matris av varje atom. Tar atomtoken (siffran för att beskriva atomen) och beskriver med en one hot vektor. Vi måste skriva om de diskreta token på detta sätt för modellen
    pos      = reshape(hcat(coord_seq...), 3, 1, length(coord_seq))  # Ändrar lite format på koordinaterna [3, B=1, T]

    μ, σ, logw, logits = model(pos, atom_ids) # skicka in atom id och pos och returnera de värden som behövs för att beräkna förlust 

    disp = pos[:, :, 2:end] .- pos[:, :, 1:end-1] # Beräkna förskjutningar mellan på varandra följande tokens. Detta är då inte strikt atom 5-4 utan följer den ordning vi skapat. Dvs en gren i taget. 

    # Maskera bort displacements vid push/pop
    coord_mask_arr = reshape(coord_mask, 1, 1, length(coord_mask)) # reshapea
    disp .= disp .* coord_mask_arr[:, :, 2:end] # vektormultiplikation så vi får 0 för de falska och inte ska räknas med sedan 

    # Koordinat-förlust via MOG
    logp_xyz = logpdf_MOG(disp, μ, σ, logw) # Log-sannolikhet för dessa displacement enligt modellens Gauss-komponenter
    loss_xyz = -sum(logp_xyz .* coord_mask_arr[:, :, 2:end]) / sum(coord_mask) # maska bort false och ta negativ medelvärde

    # Atomtyp-förlust, ignorera anchors
    target_atoms   = atom_seq[2:end]
    atom_onehot    = Flux.onehotbatch(target_atoms, 1:size(logits,1))
    atom_mask_arr  = reshape(atom_mask, 1, length(atom_mask))

    masked_mean(p) = sum(p .* atom_mask_arr[:, 2:end]) / sum(atom_mask) # maska bort false och ta negativ medelvärde
    loss_type      = Flux.logitcrossentropy(dropdims(logits, dims=2), atom_onehot; agg=masked_mean)

    return loss_xyz + loss_type # slå ihop loss 
end

# Stub för log-sannolikhet av Mixture of Gaussians
typealias_VEC SVector{3,Float32}
function logpdf_MOG(disp, μ, σ, logw)
    return sum(logpdf.(MvNormal.(μ, σ), eachslice(disp, dims=3))) .+ logw
end

In [ ]:
function loss(model, atom_seq, coord_seq, climb_seq, atom_mask, coord_mask)
    # === Format model inputs ===
    atom_ids = Flux.onehotbatch(atom_seq, 1:model.vocab_size)
    pos      = reshape(hcat(coord_seq...), 3, 1, length(coord_seq))  # [3, B=1, T]

    # === Forward pass ===
    μ, σ, logw, logits, climb_logits = model(pos, atom_ids)

    # === Coordinate loss (Mixture of Gaussians) ===
    disp = pos[:, :, 2:end] .- pos[:, :, 1:end-1]
    coord_mask_arr = reshape(coord_mask, 1, 1, length(coord_mask))
    disp .= disp .* coord_mask_arr[:, :, 2:end]

    logp_xyz = logpdf_MOG(disp, μ, σ, logw)
    loss_xyz = -sum(logp_xyz .* coord_mask_arr[:, :, 2:end]) / sum(coord_mask)

    # === Atom type loss ===
    target_atoms   = atom_seq[2:end]
    atom_onehot    = Flux.onehotbatch(target_atoms, 1:size(logits,1))
    atom_mask_arr  = reshape(atom_mask, 1, length(atom_mask))
    masked_mean(p) = sum(p .* atom_mask_arr[:, 2:end]) / sum(atom_mask)
    loss_type      = Flux.logitcrossentropy(dropdims(logits, dims=2), atom_onehot; agg=masked_mean)

    # === Climb prediction loss ===
    target_climbs = climb_seq[2:end]
    climb_onehot  = Flux.onehotbatch(target_climbs, 0:size(climb_logits, 1)-1)
    loss_climb    = Flux.logitcrossentropy(dropdims(climb_logits, dims=2), climb_onehot; agg=masked_mean)

    # === Total loss ===
    return loss_xyz + loss_type + loss_climb
end
